In [1]:
val sparksession=new SparkSession.Builder().appName("ratingsession").getOrCreate()

sparksession = org.apache.spark.sql.SparkSession@2f74ed88


org.apache.spark.sql.SparkSession@2f74ed88

In [2]:
val ratings_csv_path="gs://movielens_dataset_rd/ratings.csv"
val ratingsDfFull=sparksession.read.option("header", "true").option("inferSchema", "true").csv(ratings_csv_path)


[userId: int, movieId: int ... 2 more fields]

ratings_csv_path = gs://movielens_dataset_rd/ratings.csv
ratingsDfFull = [userId: int, movieId: int ... 2 more fields]


In [3]:
val ratingsDf=ratingsDfFull.limit(10000)

ratingsDf = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [6]:
println(ratingsDf.columns.toList)

List(userId, movieId, rating, timestamp)


In [4]:
val ratingsRdd=ratingsDf.rdd

ratingsRdd = MapPartitionsRDD[18] at rdd at <console>:24


MapPartitionsRDD[18] at rdd at <console>:24

In [18]:
//printing the schema to get the Data Type
ratingsDf.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



lastException = null


null

In [5]:
println(ratingsRdd.columns.toList)

Unknown Error: <console>:25: error: value columns is not a member of org.apache.spark.rdd.RDD[org.apache.spark.sql.Row]
       println(ratingsRdd.columns.toList)
                          ^


In [19]:
val userIdRatingRdd=ratingsRdd.map(row=>(row.getAs[Integer]("userId"),row.getAs[Integer]("movieId"),row.getAs[Double]("rating")))

userIdRatingRdd = MapPartitionsRDD[33] at map at <console>:24


MapPartitionsRDD[33] at map at <console>:24

In [20]:
val userIdRatingDF=userIdRatingRdd.toDF("userId","movieId","rating")

userIdRatingDF = [userId: int, movieId: int ... 1 more field]


[userId: int, movieId: int ... 1 more field]

In [21]:
//show the DF with only userId ,movieId and rating
userIdRatingDF.limit(1).show()


+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     17|   4.0|
+------+-------+------+



In [23]:
//grouping cells by key
val groupedByUserIdRdd=userIdRatingRdd.groupBy(x=>x._1)


groupedByUserIdRdd = ShuffledRDD[41] at groupBy at <console>:25


ShuffledRDD[41] at groupBy at <console>:25

In [24]:
println(groupedByUserIdRdd.first())

(13,CompactBuffer((13,6,5.0), (13,11,3.0), (13,25,3.0), (13,86,3.0), (13,110,5.0), (13,161,4.0), (13,225,4.0), (13,296,2.0), (13,318,5.0), (13,356,5.0), (13,357,3.0), (13,361,3.0), (13,377,3.0), (13,380,4.0), (13,457,4.0), (13,475,4.0), (13,527,4.0), (13,539,4.0), (13,587,4.0), (13,589,5.0), (13,593,4.0), (13,597,3.0), (13,733,3.0), (13,912,4.0), (13,920,4.0), (13,969,4.0), (13,1036,4.0), (13,1088,4.0), (13,1221,4.0), (13,1233,4.0), (13,1240,4.0), (13,1266,2.0), (13,1270,3.0), (13,1271,3.0), (13,1307,4.0), (13,1343,4.0), (13,1370,4.0), (13,1500,3.0), (13,1608,4.0), (13,1610,4.0), (13,1617,3.0), (13,1625,3.0), (13,1674,3.0), (13,1704,5.0), (13,1721,4.0), (13,1722,3.0), (13,2058,2.0), (13,2231,2.0), (13,2268,5.0), (13,2278,3.0), (13,2359,4.0), (13,2571,2.0), (13,2762,4.0), (13,2858,4.0), (13,3101,4.0), (13,3105,4.0), (13,3147,3.0), (13,3252,4.0), (13,3499,3.0), (13,3512,4.0), (13,3551,4.0), (13,3755,3.0), (13,3836,3.0), (13,4041,5.0), (13,4661,3.0)))


In [27]:
import org.apache.spark.rdd.RDD //adding this to avoid not fund:type RDD error
import org.apache.spark.SparkContext

//writing the groupedValues to separate files based on userId
val headers = "userId,movieId,rating"
groupedByUserIdRdd.collect().foreach { case (userId,ratingDetails) =>
  // Define the path for each user (this will create a directory for each userId)
val outputDir = s"hdfs://cluster-abbd-m.us-central1-c.c.composite-stage-440104-u5.internal.:8020/output/caseStudy2/WithRdds/user-data/$userId/ratings.csv"

  // Convert the ratings to a format suitable for saving as text (i.e., (userId, rating))
  val userRdd: RDD[String] = sc.parallelize(ratingDetails.map(rating => s"$userId,${rating._1},${rating._2}").toSeq) //converting to seq as parallelize accepts sequence and not iterable

  // Add headers as the first record
  val userWithHeaderRdd = sc.parallelize(Seq(headers)) union userRdd

  // Save the data for each userId to a separate file/directory
  userWithHeaderRdd.coalesce(1).saveAsTextFile(outputDir)
}

headers = userId,movieId,rating


userId,movieId,rating

In [ ]:
//doing it using Dataframes now
val outputDir = s"hdfs://cluster-abbd-m.us-central1-c.c.composite-stage-440104-u5.internal.:8020/output/caseStudy2/WithDataframes/user-data"

ratingsDf.write
  .partitionBy("userId")  // Partition the data based on userId
  .format("csv")      // You can use other formats like parquet, json, etc.
  .save(outputDir)